# Generate Website

This notebook generates the website for the dataset.

In [1]:
%reload_ext autoreload
%autoreload 2

## Generate JSON Reports

In [3]:
from depsurf import DepKind, VersionGroup, DepReport, LinuxImage
from typing import Iterator


GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH, VersionGroup.FLAVOR]

DEPS = [
    DepKind.FUNC("__blk_account_io_done"),
    DepKind.FUNC("__page_cache_release"),
    DepKind.STRUCT("task_struct"),
    DepKind.STRUCT("fs_context"),
    DepKind.STRUCT("inode"),
    # DepKind.FUNC("vfs_rename"),
    # DepKind.FUNC("init_once"),
    # DepKind.FUNC("vfs_read"),
    # DepKind.STRUCT("xfrm_policy_afinfo"),
    # DepKind.STRUCT("xfrm_sec_ctx"),
    DepKind.STRUCT("address_space_operations"),
]

# dep = DEPS[-1]
# DepReport.from_groups(dep, GROUPS).dump_json(dep.report_json_path)
# DepReport.from_dump(dep.report_json_path)

In [ ]:
KINDS = [
    DepKind.FUNC,
    DepKind.STRUCT,
    DepKind.LSM,
    DepKind.TRACEPOINT,
]


def get_deps_names(img: LinuxImage, kind: DepKind) -> Iterator[str]:
    if kind == DepKind.FUNC:
        return (func.name for func in img.func_groups.iter_funcs())
    elif kind == DepKind.STRUCT:
        return iter(img.struct_types.data)
    elif kind == DepKind.LSM:
        return iter(img.lsm_hooks)
    elif kind == DepKind.TRACEPOINT:
        return iter(img.tracepoints.data)
    else:
        raise ValueError(f"Unknown kind: {kind}")


DEPS = sorted(
    set(
        kind(name)
        for group in GROUPS
        for v in group
        for kind in KINDS
        for name in get_deps_names(v.img, kind)
    )
)

for dep in DEPS:
    report = DepReport.from_groups(dep, GROUPS)
    report.dump_json(dep.report_json_path)

## Generate Markdown Reports

In [1]:
from depsurf import DepReport, DepKind, WEBSITE_PATH

KINDS = [
    # DepKind.FUNC,
    # DepKind.STRUCT,
    # DepKind.LSM,
    DepKind.TRACEPOINT,
]


for kind in KINDS:
    paths = (WEBSITE_PATH / kind).rglob("*.json")
    for json_path in paths:
        report = DepReport.from_dump(json_path)
        report.dump_md(report.dep.report_md_path)

## Generate Index Pages

In [14]:
from typing import Iterable
from pathlib import Path

from depsurf import DepKind, VersionGroup, DepReport, WEBSITE_PATH, IssueEnum
from typing import TextIO


KINDS = [
    DepKind.FUNC,
    DepKind.STRUCT,
    DepKind.LSM,
    DepKind.TRACEPOINT,
]

GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH, VersionGroup.FLAVOR]

ISSUE_SYMBOLS = {
    IssueEnum.ABSENT: "X",
    IssueEnum.CHANGE: "Δ",
    IssueEnum.SELECTIVE_INLINE: "S",
    IssueEnum.FULL_INLINE: "F",
    IssueEnum.DUPLICATE: "D",
    IssueEnum.COLLISION: "C",
    IssueEnum.TRANSFORMATION: "T",
}


def print_row(report_path: Path, file: TextIO):
    report = DepReport.from_dump(report_path)
    dep = report.dep
    row = [
        "",
        f"[`{dep.name}`]({dep.report_url})",
        *(
            ",".join([ISSUE_SYMBOLS[e] for e in issues])
            for issues in report.issues_dict.values()
        ),
        "",
    ]
    print("|".join(row), file=file)


def print_header(file: TextIO):
    col_names = [g.to_str(v) for g in GROUPS for v in g]
    print("| Name | " + " | ".join(col_names) + " |", file=file)
    print("| " + " | ".join(["-"] * (len(col_names) + 1)) + " |", file=file)


def print_index(report_paths: Iterable[Path], file_path: Path):
    with open(file_path, "w") as f:
        kind = file_path.parent.name
        print(f"# {kind}", file=f)
        print_header(f)
        for report_path in sorted(
            report_paths, key=lambda d: d.stem.replace("_", "").lower()
        ):
            print_row(report_path, f)
    print(f"Saved {file_path}", flush=True)


for kind in KINDS:
    kind_path = WEBSITE_PATH / str(kind)
    subdirs = [d for d in kind_path.iterdir() if d.is_dir()]
    if kind in [DepKind.FUNC, DepKind.STRUCT]:
        for subdir in subdirs:
            print_index(
                report_paths=subdir.glob("*.json"),
                file_path=kind_path / f"{subdir.stem}.md",
            )
    else:
        print_index(
            report_paths=[p for d in subdirs for p in d.glob("*.json")],
            file_path=kind_path / "index.md",
        )

Saved /Users/szhong/Code/DepSurf/data/website/Function/r.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/u.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/i.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/n.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/g.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/z.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/t.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/s.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/a.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/f.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/o.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/h.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/m.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/j.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/c.md
Saved /Users/szhong/Code/DepSurf/data/website/Function/d.md
Saved /Users/szhong/Code/DepSurf/data/we

In [5]:
from depsurf import IssueEnum, VersionGroup

from collections import defaultdict

partitioned_results = defaultdict(dict)

for report in FUNC_REPORTS:
    result = {}
    for (group, version), issues in report.issues_dict.items():
        v = version.short_version
        if IssueEnum.FULL_INLINE in issues:
            result[v] = "F"
        elif IssueEnum.SELECTIVE_INLINE in issues:
            result[v] = "S"
        elif IssueEnum.ABSENT in issues:
            result[v] = "-"
        else:
            result[v] = ""

    dep = report.dep
    key = dep.name.replace("_", "").lower()
    partitioned_results[key[:1]][(key, dep.name)] = result


partitioned_results = {k: sorted(v.items()) for k, v in partitioned_results.items()}

In [6]:
INLINED_PATH = WEBSITE_PATH / "inline"
INLINED_PATH.mkdir(exist_ok=True)

for char, results in partitioned_results.items():
    with open(INLINED_PATH / f"{char}.html", "w") as f:
        f.write("<html>\n")
        f.write("<head>\n")
        f.write(f"<title>Kernel Function Inline Status - {char}</title>\n")
        f.write('<link rel="stylesheet" href="style.css">\n')
        f.write("</head>\n")

        f.write("<body>\n")
        f.write("<table>\n")
        f.write("<thead>\n")

        f.write("<tr>\n")
        f.write(
            f'<td><span class="f">&nbsp;F&nbsp;</span>: Fully Inlined. <span class="p">&nbsp;P&nbsp;</span>: Partially Inlined. <span class="m">&nbsp;-&nbsp;</span>: Absent</td>\n'
        )
        for version in VersionGroup.REGULAR:
            f.write(f"<th>{version.short_version}</th>")
        f.write("</tr>\n")
        f.write("</thead>\n")

        f.write("<tbody>\n")
        for (_, dep), result in results:
            f.write(f'<tr><th id="{dep}"><pre>{dep}</pre></th>')
            for v in result.values():
                class_name = {
                    "F": "f",
                    "P": "p",
                    "-": "m",
                }.get(v)
                if class_name is not None:
                    f.write(f'<td class="{class_name}">{v}</td>')
                else:
                    f.write(f"<td>{v}</td>")
            f.write("</tr>\n")
        f.write("</tbody>\n")

        f.write("</table>\n")
        f.write("</body>\n")
        f.write("</html>\n")